## Run Simulation Trials

The run script is the main entrypoint for this project. It has a number of options that can be dynamically set for a wide range of simulation scenarios. This are available for reference by passing the help flag as a parameter.

In [2]:
!./run.sh -h

./run.sh usage:
        -a  Select path planning algorithm/strategy (pso, aco or lawn).
        -b  Build the main argos project. Use after editing source files.
        -d  Set the number of drones to place in simulation.
        -e  Set experiment source file. Currently defaults to "main".
        I)  Run installation of environment packages and dependancies.
        -j  Run the jupyter environment.
        -n  Set number of targets/plants to place in simulation.
        N)  Set value range of targets/plants to place in simulation.
        -s  Set the number of independantly seeded trials to run.
        -t  Set the target coverage/inspection percentage during trial.
        -v  Enable argos vizualization. Disabled by default for speed.
        h | *)  Print this usage info.


Let's create a dataset with the following settings:
- Target Coverage Percentage of 90%
- Across all three path planning algorithms/strategies.
- Independantly seeding 3 random child trials per parent iteration.
- With a target/plant value range of 4.

In [5]:
!sudo -S ./run.sh -t "0.90" -a "pso aco" -s 3 -N "4" < pass

[sudo] password for jo: Building and making project...
-- Could NOT find GALIB (missing:  GALIB_LIBRARY GALIB_INCLUDE_DIR) 
-- Configuring done
-- Generating done
-- Build files have been written to: /mnt/d/Linux/thesis/source2/build
[  4%] Automatic moc for target main
[  4%] Built target main_automoc
[  8%] Automatic moc for target pso
[  8%] Built target pso_automoc
[ 39%] Built target pso
[ 43%] Automatic moc for target aco
[ 43%] Built target aco_automoc
[ 69%] Built target aco
[ 73%] Automatic moc for target kalman
[ 73%] Built target kalman_automoc
[ 86%] Built target kalman
[100%] Built target main
Done!
Beginning pso trial with ( targets = 1 )( threshold = 0.90 )( seed = 3722970741 )
Running main experiment...
Done!
( threshold = 0.90 )( seed = 4289828130 )
Running main experiment...
Done!
( threshold = 0.90 )( seed = 2905981271 )
Running main experiment...
Done!
( targets = 2 )( threshold = 0.90 )( seed = 119104093 )
Running main experiment...
Done!
( threshold = 0.90 )( seed

## Testing for Significance with the Independant Students' T-test

We will use the Independant Student's t-test to determine whether the difference between our **UAV-MRS** simulation datasets is significant. We first import the necessary python packages.

In [244]:
import pandas as pd
from scipy import stats
from math import sqrt
from scipy.stats import ttest_ind
from scipy.stats import t
import numpy as np

We then load our dataset in chunks to categorised pandas dataframes.

In [251]:
chunksize = 10 ** 4
filename = 'output/data_0.csv'
headers = ['Type','TargetNum','TargetThresh','Step','Completed',
         'X','Y','Z','RtMProb','RtLProb','MinimumHold','LaunchStep',
         'InitialRtMProb','RtMDelta','InitialRtLProb','RtLDelta',
         'MinimumRest','InitialMinimumHold','MaximumHold','GlobalReach',
         'ProximityThresh','Attitude','SwarmParticles','SwarmSelfTrust',
         'SwarmPastTrust','SwarmGlobalTrust','SwarmAnts','MappingMean',
         'MappingStdDev','MappingSeed','RtMMin','RtMMax','RtMSeed','RtLMin',
         'RtLMax','RtLSeed','ACOSeed','TaskCompletedMin','TaskCompletedMax',
         'TaskCompletedSeed','TargetShuffleMin','TargetShuffleMax',
         'TargetShuffleSeed','NaiveMapping','VStep','HStep','ArgosSeed']

def getCategories(chunk):
    return (chunk.loc[chunk['Type'] == 'pso'], chunk.loc[chunk['Type'] == 'aco'], chunk.loc[chunk['Type'] == 'lawn'])

pso_f = pd.DataFrame(columns=headers, dtype=float)
aco_f = pd.DataFrame(columns=headers, dtype=float)
lawn_f = pd.DataFrame(columns=headers, dtype=float)

for chunk in pd.read_csv(filename, chunksize=chunksize):
    (pso_c, aco_c, lawn_c) = getCategories(chunk)
    pso_f = pso_f.append(pso_c, ignore_index = True)
    aco_f = aco_f.append(aco_c, ignore_index = True)
    lawn_f = lawn_f.append(lawn_c, ignore_index = True)

KeyboardInterrupt: 

Compute the mean across all seeded child trials.

In [ ]:
pso_target_thresh = pso_f.groupby('TargetNum', as_index=False).apply(lambda row: row[row['Completed'] == row['TargetThresh']])
aco_target_thresh = aco_f.groupby('TargetNum', as_index=False).apply(lambda row: row[row['Completed'] == row['TargetThresh']])
lawn_target_thresh = lawn_f.groupby('TargetNum', as_index=False).apply(lambda row: row[row['Completed'] == row['TargetThresh']])

pso_step_means = pso_target_thresh.groupby('ArgosSeed').head(1).groupby('TargetNum')['Step'].mean()
aco_step_means = aco_target_thresh.groupby('ArgosSeed').head(1).groupby('TargetNum')['Step'].mean()
lawn_step_means = lawn_target_thresh.groupby('ArgosSeed').head(1).groupby('TargetNum')['Step'].mean()

means = pd.DataFrame(columns=['pso', 'aco', 'lawn'])

means.pso = pso_step_means
means.aco = aco_step_means
means.lawn = lawn_step_means

print(means)

Perform independant student t-test

In [ ]:
print('T-test of pso and aco means: ', ttest_ind(means.pso,means.aco))
print('T-test of pso and lawn means: ', ttest_ind(means.pso,means.lawn))
print('T-test of aco and lawn means: ', ttest_ind(means.aco,means.aco))